<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/TextCNN_with_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important sorces:
https://chatgpt.com/share/21220a6d-4cca-48bc-b461-bca6b0e140bc

In [ ]:
pip install gensim

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
file_path = './saved_file'
file_path = ('ARPs_and_ProgrammingPosts.xlsx')
arp_data = pd.read_excel(file_path, sheet_name='13,205 ARPs')

In [7]:
# Parameters
max_len = 100  # Maximum length of all sequences

In [8]:
# Extract the text and labels
texts = arp_data['Question_body'].astype(str).tolist()
labels = arp_data['Label'].tolist()

In [9]:
# Tokenize the text and train Word2Vec model
tokenized_texts = [text.split() for text in texts]
word2vec_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)


In [11]:
# Create an embedding matrix
vocab_size = len(word2vec_model.wv.key_to_index) + 1
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

word_index = {word: index for index, word in enumerate(word2vec_model.wv.index_to_key, 1)}

for word, index in word_index.items():
    embedding_matrix[index] = word2vec_model.wv[word]

In [12]:
# Convert text to sequences of word indexes
sequences = [[word_index.get(word, 0) for word in text] for text in tokenized_texts]


In [13]:
# Pad the sequences to ensure uniform input size
data = pad_sequences(sequences, maxlen=max_len)

In [14]:
# Convert labels to numpy array
labels = tf.keras.utils.to_categorical(labels, num_classes=2)

In [15]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [16]:
# Define the neural network model
def create_model(vocab_size, embedding_dim, input_length, embedding_matrix):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  output_dim=embedding_dim,
                                  weights=[embedding_matrix],
                                  input_length=input_length,
                                  trainable=False),
        tf.keras.layers.Conv1D(128, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# Create the model
model = create_model(vocab_size, embedding_dim, max_len, embedding_matrix)


In [18]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
299/299 [==============================] - 11s 33ms/step - loss: 0.3207 - accuracy: 0.8744 - val_loss: 0.2668 - val_accuracy: 0.8907
Epoch 2/10
299/299 [==============================] - 8s 26ms/step - loss: 0.2248 - accuracy: 0.9102 - val_loss: 0.3159 - val_accuracy: 0.8882
Epoch 3/10
299/299 [==============================] - 9s 31ms/step - loss: 0.2007 - accuracy: 0.9165 - val_loss: 0.2799 - val_accuracy: 0.8761
Epoch 4/10
299/299 [==============================] - 9s 31ms/step - loss: 0.1707 - accuracy: 0.9310 - val_loss: 0.2944 - val_accuracy: 0.8849
Epoch 5/10
299/299 [==============================] - 8s 25ms/step - loss: 0.1550 - accuracy: 0.9385 - val_loss: 0.3389 - val_accuracy: 0.8928
Epoch 6/10
299/299 [==============================] - 10s 32ms/step - loss: 0.1184 - accuracy: 0.9520 - val_loss: 0.3925 - val_accuracy: 0.8255
Epoch 7/10
299/299 [==============================] - 8s 27ms/step - loss: 0.0888 - accuracy: 0.9667 - val_loss: 0.3807 - val_accuracy: 0.85

In [19]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


94/94 [==============================] - 1s 13ms/step - loss: 0.4563 - accuracy: 0.8664
Test Accuracy: 0.8664


In [20]:
# Make predictions
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)


94/94 [==============================] - 1s 9ms/step


In [21]:
# Calculate precision, recall, and F1-score
report = classification_report(y_true_classes, y_pred_classes, target_names=['Class 0', 'Class 1'])
print(report)

              precision    recall  f1-score   support

     Class 0       0.85      0.89      0.87      1504
     Class 1       0.89      0.84      0.86      1483

    accuracy                           0.87      2987
   macro avg       0.87      0.87      0.87      2987
weighted avg       0.87      0.87      0.87      2987



In [22]:

# Save the model
model.save('word2vec_textcnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
